# Nonlinear Least Squares curve fitting
Trying to put together a minimal implementation of the Levenberg-Marquardt algorithm for nonlinear least squares curve fitting.
This is mostly for my own learning, but it was also requested on [my blog](https://workyourtech.com/2020/04/05/fitting-curves-to-data-with-an-arduino/#comment-28)

## Following these papers: [this](http://people.duke.edu/~hpgavin/ce281/lm.pdf) and [this](https://www.eecs189.org/static/notes/n12.pdf)

In [1]:
using Plots
using LinearAlgebra # For I

## Chi square measure of goodness of fit (used for λ measure in LM algo)

In [2]:
function chisquare(O, E)
    return sum(((O .- E).^2) ./ E)
end

chisquare (generic function with 1 method)

## The nicest(?) way I've found to calculate the jacobian matrix is by generating a square matrix with each parameter perturbation in a row like this, then map each row of the perturbation matrix to get mxn matrix

In [13]:
function perturbparameters(parameters, perturbation)
    return parameters .* (ones(length(parameters), length(parameters)) + perturbation * I)
end
perturbparameters([1, 2, 3], 0.001)

3×3 Array{Float64,2}:
 1.001  1.0    1.0
 2.0    2.002  2.0
 3.0    3.0    3.003

## Jacobian function
The functions we're working with here take a vector of datapoints of **m** length and return a vector of the function applied to each of those datapoints.

The function takes **n** parameters.

The jacobian is an **m**x**n** matrix that gives the gradient of each parameter if all the others are fixed, for all **n** datapoints.

In [ ]:
function jacobian(f, x::Vector{Float64}, parameters, perturbation=0.0001)::Array{Float64, 2}
    return cat(
        map(row -> (f(x, row...) - f(x, parameters...)) / perturbation,
            eachrow(perturbparameters(parameters, perturbation)))...,
        dims=2)
end

## Meat of the algo, two different update functions, gradient descent and gauss-newton

In [3]:
function gradient(f, x, y, ŷ, parameters, J)
    return transpose(J)*(y-ŷ)
end

function gaussnewton(f, x, y, ŷ, parameters, J)
    return inv(transpose(J)*J)*transpose(J)*(y-ŷ)
end

# Todo: Use Broyden rank-1 update for updating jacobian in certain conditions
# http://people.duke.edu/~hpgavin/ce281/lm.pdf
# https://www.eecs189.org/static/notes/n12.pdf

function step(f, x::Vector{Float64}, y::Vector{Float64}, parameters, α=0.0001)
    W = I
    p = copy(parameters)
    iterations = 1
    while true
        ŷ = f(x, p...)
        J = jacobian(f, x, parameters)
        grad = gaussnewton(f, x, y, ŷ, p, J)
        p = p + grad
        sumsquares = sum((y-ŷ).^2)
        println("Experiment new parameters $(p) loss: $(sumsquares)")
        iterations = iterations + 1
        iterations > 50 && break
        sumsquares > 1 || break
    end
    return p
end

step (generic function with 2 methods)

In [4]:
α = 0.01
f(x, a, b, c, d, e) = a.+b.*exp.(-c.*x) .* sin.(d.*x.+e)
x = collect(0:0.1:3)
y = f(x, 5, 6, 7, 8, 9)
p = [1, 1, 1, 1, 1]

5-element Array{Int64,1}:
 1
 1
 1
 1
 1

In [5]:
newparameters = gradientdescent(f, collect(x), y, p, α)
ŷ = f(x, newparameters...)
plot([x, x], [ŷ, y])

UndefVarError: [91mUndefVarError: gradientdescent not defined[39m

In [6]:
chisquare(y, ŷ)

UndefVarError: [91mUndefVarError: ŷ not defined[39m

In [7]:
print(newparameters)

UndefVarError: [91mUndefVarError: newparameters not defined[39m